In [328]:
import numpy as np
from scipy.sparse.linalg import LinearOperator, svds
from scipy.linalg import svd

from pysgpp import DataMatrix, DataVector, createOperationMultipleEval, Grid
from __future__ import division

import sys, os

sys.path.append(os.path.abspath(os.path.join('..', 'datasets/')))
import tools

In [2]:
grid = Grid.createLinearGrid(4)
gen = grid.getGenerator()
storage = grid.getStorage()
gen.regular(5)

In [3]:
def eval_op(x, op, size):
    result_vec = DataVector(size)
    x = DataVector(x.flatten())
    op.mult(x, result_vec)
    return result_vec.array().copy()

def eval_op_transpose(x, op, size):
    result_vec = DataVector(size)  
    x = DataVector(x.flatten())
    op.multTranspose(x, result_vec)
    return result_vec.array().copy()


data = np.random.random((10000,4))
data_set = tools.to_data_matrix(data)

num_elem = data.shape[0]

op = createOperationMultipleEval(grid, data_set)
matvec = lambda x: eval_op(x, op, num_elem)
rmatvec = lambda x: eval_op_transpose(x, op, grid.getSize())
shape = (num_elem, grid.getSize())
linop = LinearOperator(shape, matvec, rmatvec, dtype='float64')

In [4]:
u, s, v = svds(linop, k=10)
s

array([ 11.17666591,  12.04285912,  12.23893054,  12.28622521,
        12.53643802,  16.98986957,  17.45206982,  17.56760379,
        17.76053682,  27.12652157])

#FISTA

## Proximal operator & splitting

(see proximal algorithms boyd)

$$
f(x) = (1/2)\Vert Ax - b \Vert_2^2, \qquad g(x) = \lambda \Vert x \Vert_1 \\
\nabla f(x) = A^\text{T}(Ax-b) \\
\text{prox}_{\lambda g}(x) = ( x - \lambda)_+ - (-x - \lambda)_+
\\
$$

In [355]:
def proximal_l1(x, alpha):
    return np.multiply(np.maximum(np.abs(x) - alpha, np.zeros(x.shape)), np.sign(x))

def gradient(A, x, b):
    return A.transpose() * (A * x - b)

def get_stepsize(A):
    singular_values = svd(A, compute_uv=False)
    max_sval_A = singular_values.max()
    max_eigen_AA_t = max_sval_A * max_sval_A
    return 1.0/(2*max_eigen_AA_t)

def get_residual(A, x, b):
    res = A * x - b
    res = np.multiply(res, res)
    return res.sum()

def ista(A, x_start, num_it, lambda_reg, b):
    x_k = x_start
    stepsize = get_stepsize(A)
    for i in xrange(1,num_it+1):
        x_k = proximal_l1(x_k - stepsize * gradient(A, x_k, b), lambda_reg * stepsize)
        if i % (num_it/10) == 0:
            residual = get_residual(A, x_k, b)
            print "Iteration {}\t with residual of {}".format(i, residual)
    return x_k

def fista(A, x_start, num_it, lambda_reg, b):
    stepsize = get_stepsize(A)
    x_k = x_start
    y_k = x_k.copy()
    t_k = 1.0
    for i in xrange(1,num_it+1):
        x_before = x_k.copy()
        x_k = proximal_l1(y_k - stepsize * gradient(A, y_k, b), lambda_reg * stepsize)
        
        t_before = t_k
        t_k = (1 + np.sqrt(1 + 4 * t_before**2))/2
        
        y_k = x_k + ((t_before - 1)/(t_k)) * (x_k - x_before)
        if i % (num_it/10) == 0:
            residual = get_residual(A, x_k, b)
            print "Iteration {}\t with residual of {}".format(i, residual)
    return x_k

In [389]:
A = np.matrix(np.random.rand(1000,800))
x = np.matrix(np.zeros((800,1)))
b = np.matrix(np.random.rand(1000,1))
lambda_reg = 0.0001

In [390]:
2*svd(A, compute_uv=False).max()**2

400655.84587203903

In [391]:
x_ista = ista(A, x, 10000, lambda_reg, b)

Iteration 1000	 with residual of 61.1701624071
Iteration 2000	 with residual of 50.8026638102
Iteration 3000	 with residual of 44.7344170053
Iteration 4000	 with residual of 40.7795080469
Iteration 5000	 with residual of 37.9856640046
Iteration 6000	 with residual of 35.8936490079
Iteration 7000	 with residual of 34.259370125
Iteration 8000	 with residual of 32.9415263753
Iteration 9000	 with residual of 31.8523833703
Iteration 10000	 with residual of 30.9343565576


In [392]:
x_fista = fista(A,x,10000,lambda_reg,b)

Iteration 1000	 with residual of 19.6197462981
Iteration 2000	 with residual of 18.7140978412
Iteration 3000	 with residual of 18.6768730249
Iteration 4000	 with residual of 18.6587703128
Iteration 5000	 with residual of 18.6541043312
Iteration 6000	 with residual of 18.6520234856
Iteration 7000	 with residual of 18.6502371263
Iteration 8000	 with residual of 18.6505236215
Iteration 9000	 with residual of 18.6496238376
Iteration 10000	 with residual of 18.6497038547


In [379]:
np.abs(get_residual(A, x_ista, b)) < np.abs(get_residual(A, x_fista, b))

False

In [380]:
numpy = np.linalg.lstsq(A, b)[0]
get_residual(A, numpy, b)

9638.4719014387247

In [381]:
from scipy.linalg import eig
np.abs(eig(A.transpose()*A)[0])


array([  2.01797253e+07,   2.73243337e+05,   2.65989906e+05,
         2.56084049e+05,   2.44195196e+05,   2.21074940e+05,
         2.16947226e+05,   2.09831153e+05,   1.98367715e+05,
         1.93003334e+05,   1.86350927e+05,   1.79581606e+05,
         1.71462447e+05,   1.63842880e+05,   1.57242659e+05,
         1.47566496e+05,   1.41916978e+05,   1.38376058e+05,
         1.35397613e+05,   1.32109914e+05,   1.25688277e+05,
         1.21569081e+05,   1.19030076e+05,   1.17672474e+05,
         1.10522029e+05,   1.08518068e+05,   1.06446094e+05,
         1.01040764e+05,   9.73668844e+04,   9.39739776e+04,
         9.35560143e+04,   8.95713146e+04,   8.71458351e+04,
         8.39315454e+04,   8.23346911e+04,   8.03628094e+04,
         7.49206826e+04,   7.38949819e+04,   6.89966112e+04,
         6.42724439e+04,   6.15272563e+04,   5.88272565e+04,
         5.70776098e+04,   5.48999147e+04,   5.12795282e+04,
         5.02610457e+04,   4.54832725e+04,   4.34642918e+04,
         3.93111617e+04,